In [1]:
filename = '2022-06 06 Fact 30-May al 05-Jun.xlsx'
year = 2022
week = 23

# Libraries

In [2]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format

# Parameters

In [3]:
path = r'\\NASPRO.infovisiontv.com\DGI\DGI Banco Central\00 CIERRES SEMANALES\{0}\{0} {1}'
path = path.format(year, str(week).zfill(2))

# Import Data

'Grupo de artículos'

In [4]:
%%time
#df = pd.read_excel(path + '\\' + filename, sheet_name='YOSAF', dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})
df = pd.read_excel(path + '\\' + filename, dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})

CPU times: total: 13.7 s
Wall time: 17.8 s


df[['Grupo de artículos', 'Tp.cambio p.contab.', 'Lote']].dtypes

In [5]:
df.shape

(14571, 47)

In [6]:
df.tail(10)

,Doc.fact.,Posición,Documento de ventas,Posición.1,Referencia cliente,Material,Cd.Barra,Denomin.posición,Solicitante,Nombre,...,Nombre empresa,Organización ventas,Centro,Almacén,Lote,FeCaduc/FePreferCons,Motivo pedido,Descripción Motivo Pedido,Valor neto.1,Moneda del documento
14561,"196,272,914.00",11.00,"15,324,024.00",10.00,XL3 Extra Recup 1,"8,100,001,422.00",NaN,XL3 XTRA CAJA CON 12 CAPSULAS,"1,100,034.00",REPRESENTACIONES JMP SA DE CV,...,GIBART,VGIB,MX06,MX06,2091020742,2024-04-30,NaN,NaN,"82,998.30",MXN
14562,"196,272,914.00",12.00,"15,324,024.00",10.00,XL3 Extra Recup 1,"8,100,001,422.00",NaN,XL3 XTRA CAJA CON 12 CAPSULAS,"1,100,034.00",REPRESENTACIONES JMP SA DE CV,...,GIBART,VGIB,MX06,MX06,2091020743,2024-04-30,NaN,NaN,"1,506,185.60",MXN
14563,"196,272,914.00",13.00,"15,324,024.00",10.00,XL3 Extra Recup 1,"8,100,001,422.00",NaN,XL3 XTRA CAJA CON 12 CAPSULAS,"1,100,034.00",REPRESENTACIONES JMP SA DE CV,...,GIBART,VGIB,MX06,MX06,2094020744,2024-04-30,NaN,NaN,"1,724,640.00",MXN
14564,"196,272,915.00",10.00,"15,324,028.00",10.00,Recup Faltantes Ferm,"8,100,005,255.00",650240026201,FERMODYL ENER REPAR AMPOS 4X15 ML/12,"1,100,034.00",REPRESENTACIONES JMP SA DE CV,...,GIBART,VGIB,MX06,MX06,M21Y10A,2024-11-30,NaN,NaN,"3,184.80",MXN
14565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"6,106.44",EUR
14566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"263,211,112.58",MXN
14567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"3,297,158.85",USD
14568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


# Data Cleaning

En esta línea omitimos las 6 filas nulas:

In [7]:
df = df[:df.shape[0]-6]

In [8]:
df.isnull().sum()[df.isnull().sum() > 0]

Referencia cliente             246
Cd.Barra                      3302
RFC                           3977
Nº cuenta anterior            2090
Destinatario mcía.             246
Nombre.1                       246
Descripción CeBe               246
Almacén                       1278
Lote                          1301
FeCaduc/FePreferCons          1301
Motivo pedido                13178
Descripción Motivo Pedido    13178
dtype: int64

Aquí renombraremos las columnas:

In [9]:
new_cols = [
     'Factura',
     'Posicion',
     'DocumentoVentas',
     'PosicionDV',
     'NumeroPedCliente',
     'Material',
     'CodigoBarras',
     'Denominacion',
     'Solicitante',
     'Nombre',
     'RFC',
     'CNAC',
     'GrupoCuentas',
     'GrupoTesoreria',
     'DestinatarioMcia',
     'NombreDestinatario',
     'CantidadFacturada',
     'UnidadMedida',
     'CtdFactUMB',
     'UnidadMedidaBase',
     'PrecioUnitario',
     'ImpuestoBase',
     'ImpuestoDescuento',
     'ValorNeto',
     'ImporteImpuesto',
     'ImpuestoIEPS',
     'ImpuestoTotal',
     'CentroBeneficio',
     'DescripcionCebe',
     'TipoUnidades',
     'FechaFactura',
     'GrupoArticulos',
     'DenominacionArticulos',
     'ClaseFactura',
     'DenominacionTipoDocto',
     'CambioContable',
     'Sociedad',
     'Nombreempresa',
     'OrganizacionVentas',
     'Centro',
     'Almacen',
     'Lote',
     'FechaCaducidad',
     'MotivoPedido',
     'DescMotivoPedido',
     'ValorNetoAbs',
     'Moneda'
]

In [10]:
df.columns = new_cols

Estas columnas en vez de tener nulos, deben ser 5 espacios vacío, sino se cumple esto, no se verá reflajada la data de facturación:

In [11]:
cols_nulls = ['RFC', 'CNAC', 'Almacen', 'Lote', 'MotivoPedido', 'DescMotivoPedido']
for col in cols_nulls: 
    df[col].fillna('     ', inplace=True)

In [12]:
df.isnull().sum()[df.isnull().sum() > 0]

NumeroPedCliente       246
CodigoBarras          3302
DestinatarioMcia       246
NombreDestinatario     246
DescripcionCebe        246
FechaCaducidad        1301
dtype: int64

Transformamos de decimales a enteros las siguientes columnas:

In [13]:
cols_int = [
    'Factura',
    'Posicion',
    'DocumentoVentas',
    'PosicionDV',
    'Material',
    'Solicitante',
    'CtdFactUMB',
    'ImpuestoDescuento',
    'ImpuestoIEPS',
]

In [14]:
for col in cols_int:
    df[col] = df[col].map(int)

Verificamos que estas dos columnas sean fechas (sino lo son pues hay que convertirlas):

In [15]:
df[['FechaCaducidad', 'FechaCaducidad']].dtypes

FechaCaducidad    datetime64[ns]
FechaCaducidad    datetime64[ns]
dtype: object

Verificar que las fechas de factura y caducidad tengan sentido (por ejemplo que no caduque en el año 9999):

In [16]:
print('FechaFactura desde:', df['FechaFactura'].min())
print('FechaFactura hasta:', df['FechaFactura'].max())

FechaFactura desde: 2022-05-30 00:00:00
FechaFactura hasta: 2022-06-05 00:00:00


In [17]:
print('FechaCaducidad desde:', df['FechaCaducidad'].min())
print('FechaCaducidad hasta:', df['FechaCaducidad'].max())

FechaCaducidad desde: 2022-02-09 00:00:00
FechaCaducidad hasta: 2030-03-10 00:00:00


# Export Data

Exportamos la información a la misma carpeta que la cargamos:

In [18]:
df.to_excel(path + '\\Cargar ' + filename.split('.')[0] + '.xlsx',
            index=False,
            sheet_name='carga')

In [19]:
df.shape

(14565, 47)

# Validations

In [20]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

In [21]:
qtime = '''SELECT DISTINCT 
                TmpFecha,
                TmpSemanaAnioGenomma AS SemanaGL,
                TmpAnioSemanaGenomma AS AnioGL
            FROM Gnm_DWH.dbo.Dim_Tiempo'''

In [22]:
dtime = pd.read_sql(qtime, conn)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
df = df.merge(dtime, left_on='FechaFactura', right_on='TmpFecha', how='left')

In [24]:
df.pivot_table(
    index='SemanaGL', 
    values=['ValorNeto'],
    aggfunc='sum'
)

,ValorNeto
SemanaGL,
23,"213,123,805.19"


In [25]:
df.pivot_table(
    index='SemanaGL', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
SemanaGL,
23,14565
All,14565


In [26]:
df.pivot_table(
    index='FechaFactura', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
FechaFactura,
2022-05-30 00:00:00,2679
2022-05-31 00:00:00,3308
2022-06-01 00:00:00,2147
2022-06-02 00:00:00,2075
2022-06-03 00:00:00,1100
2022-06-04 00:00:00,1565
2022-06-05 00:00:00,1691
All,14565


In [27]:
df.pivot_table(
    index='MotivoPedido', 
    values='ValorNeto', 
    aggfunc='sum'
)

,ValorNeto
MotivoPedido,
,"251,913,448.70"
M03,"-1,824.00"
M05,"-91,688.67"
M07,"-243,921.68"
M08,"-664,446.81"
M09,"-368,501.17"
M13,"-71,503.56"
M14,"-44,580.32"
M19,"-68,536.32"
